### 1. Структура базы данных

In [ ]:
Table Transactions {
  transaction_id serial [primary key]
  customer_id integer
  product_id integer
  transaction_date date
  online_order bool
  order_status varchar 
}

Table customers {
  customer_id serial [primary key]
  first_name varchar
  last_name varchar
  date_of_birth date
  job_title varchar
  job_industry_category varchar
  wealth_segment varchar
  deceased_indicator varchar
  owns_car varchar
  address varchar
  postcode integer
  state varchar
  country varchar
  property_valuation smallint
}

Table products {
  product_id serial [primary key]
  brand varchar
  product_line varchar
  product_class varchar
  product_size varchar
  list_price numeric(20, 2)
  standard_cost numeric(20, 2)
}

Ref: Transactions.customer_id < products.product_id

Ref: Transactions.product_id < customers.customer_id

Изображение структуры БД находится в репозитории*

### 2. Нормализация БД

База данных была приведена ко второй нормальной форме. 

Изначально у нас было две таблицы: "transaction" и "customer". В таблицы "transaction" существовала функциональная зависимость между "product_id" и некоторыми другими атрибутами. Вторая нормальная форма гласит, что в таблице не должно быть другой зависимости, кроме зависимости атрибута от первичного ключа.

Поэтому было принято решение вынести в отдельные таблицы "Products" и "Customers" их id (в качестве первичного ключа) и остальные атрибуты, которые являются от этих ключей зависимыми.

### 3. Создание таблиц в DBEAVER

In [37]:
import pandas as pd

In [39]:
df_1 = pd.read_csv('asset.csv')
df_2 = pd.read_csv('prodtrans.csv')

df_1.isnull().sum()

customer_id                0
first_name                 0
last_name                125
gender                     0
DOB                       87
job_title                506
job_industry_category    656
wealth_segment             0
deceased_indicator         0
owns_car                   0
address                    0
postcode                   0
state                      0
country                    0
property_valuation         0
dtype: int64

In [40]:
df_2.isnull().sum()

transaction_id        0
product_id            0
customer_id           0
transaction_date      0
online_order        360
order_status          0
brand               197
product_line        197
product_class       197
product_size        197
list_price            0
standard_cost       197
dtype: int64

In [ ]:
CREATE TABLE "transactions" (
  "transaction_id" integer PRIMARY KEY,
  "customer_id" integer references customers(customer_id),
  "product_id" integer references products(product_id),
  "transaction_date" date not NULL,
  "online_order" bool,
  "order_status" varchar(256) not NULL
);

CREATE TABLE "customers" (
  "customer_id" integer PRIMARY KEY,
  "first_name" varchar(256) not NULL,
  "gender" varchar(16) not null, 
  "last_name" varchar(256),
  "date_of_birth" date,
  "job_title" varchar(256),
  "job_industry_category" varchar(256),
  "wealth_segment" varchar(256) not NULL,
  "deceased_indicator" varchar(16) not NULL,
  "owns_car" varchar(16) not NULL,
  "address" varchar(256) not NULL,
  "postcode" integer not NULL,
  "state" varchar(32) not NULL,
  "country" varchar(32) not NULL,
  "property_valuation" smallint not NULL
);

CREATE TABLE "products" (
  "product_id" integer PRIMARY KEY,
  "brand" varchar(256),
  "product_line" varchar(256),
  "product_class" varchar(16),
  "product_size" varchar(16),
  "list_price" numeric(20,2) not NULL,
  "standard_cost" numeric(20,2)
);


### 4. Заполнение таблицы данными

Данные с датой исказились при импорте, переведем их к типу date

In [ ]:
ALTER TABLE asset ALTER column "DOB" TYPE DATE 
using to_date("DOB", 'YYYY-MM-DD')

ALTER TABLE prodtrans ALTER column "transaction_date" TYPE DATE 
using to_date("transaction_date", 'MM/DD/YYYY')

In [ ]:
UPDATE prodtrans 
SET customer_id = 4000
WHERE customer_id = 5034

insert into "products" (product_id, brand , product_line , product_class , product_size , list_price, standard_cost)
select distinct on (product_id) product_id, brand , product_line  , product_class  , product_size  , list_price , standard_cost  from "prodtrans" 

insert into "customers" (customer_id, 
                        first_name, 
                            gender, 
                        last_name, d
                        ate_of_birth, 
                        job_title, 
                        job_industry_category, 
                        wealth_segment, 
                        deceased_indicator, 
                        owns_car, 
                        address, 
                        postcode, 
                        state, 
                        country, 
                        roperty_valuation)
select customer_id, first_name, gender, last_name, "DOB", job_title, job_industry_category, wealth_segment, deceased_indicator, owns_car, address, postcode, state, country, property_valuation from "asset"


insert into "transactions" (transaction_id, customer_id, product_id, transaction_date, online_order, order_status)
select transaction_id, customer_id, product_id, transaction_date, online_order, order_status from "prodtrans" 

Один из customer_id вызывал ошибку, поэтому пришлось его просто заменить на другой